In [ ]:
import pandas as pd
import numpy as np
import charset_normalizer
import fuzzywuzzy
from fuzzywuzzy import process

In [ ]:
# for Box-Cox Transformation
from scipy import stats

In [ ]:
# for min_max scaling
from mlxtend.preprocessing import minmax_scaling

In [ ]:
# plotting modules
import seaborn as sns

In [ ]:

import matplotlib.pyplot as plt

In [ ]:
# set seed for reproducibility
np.random.seed(0)


check  what type of encoding is of file if it is not UTF-8

In [ ]:
with open("Building_Permits.csv", 'rb') as rawdata:
    result = charset_normalizer.detect(rawdata.read(1000))
    print(result)

In [ ]:
# lead scv file
permits=pd.read_csv('Building_Permits.csv')

Print first 5 lines of dataset

In [ ]:
permits.head()

# Data clean up

1. Remove null values or replace null values in data set

we are checking how many values are null in each coloum

isnull().sum() function defines if values are Nan and then num it up per coloum

In [ ]:
missing_values_count = permits.isnull().sum()
for value in missing_values_count.keys():
    if missing_values_count[value] > 0:
        print(value, missing_values_count[value])
    

In [ ]:
# how many total missing values do we have?
total_cells = np.prod(permits.shape)
total_missing = missing_values_count.sum()

# percent of data that is missing
percent_missing = (total_missing/total_cells) * 100
print(percent_missing)

remove coloums with empty values

In [ ]:
# drops coloums where at least one value is null
newPermitsone=permits.dropna(axis=1)
# drops coloums where all values are null
newPermitsall=permits.dropna(axis=1,how='all')

print('Orignal Data Set',permits.shape[1])
print('Droping coloum with atleast one null value',newPermitsone.shape[1])
print('Droping coloums where all values are null',newPermitsall.shape[1])

fill empty values with 0 and nearset value

In [ ]:
cleanedPermit=permits.bfill(axis=0).fillna(0)

# Data Scalling and Normalization

1. Scalling: this changes the range of data
2. Normalization: this transfomrs or normalize the data.Scaling just changes the range of your data. Normalization is a more radical transformation. The point of normalization is to change your observations so that they can be described as a normal distribution.

considering 'Estimated Cost','Revised Cost','Existing Units','Proposed Units' for appling normalization and scaling

for scalling we can use units to scale them between 1 to 0 so that they easy to calculate

In [ ]:
cleanedPermit.head()[['Estimated Cost','Revised Cost','Existing Units','Proposed Units']]

## Scaling

In [ ]:
# extracting coloum value and createing df 
original_data_ExistingUnits =  pd.DataFrame(cleanedPermit['Existing Units'] )
# mix-max scale the data between 0 and 1
scaled_data_ExistingUnits = minmax_scaling(original_data_ExistingUnits, columns=['Existing Units'] )

as seen in graph whole data set is not sclaaed between 0 to 1 from vast range. This does not change how data is just replace numbers so that it is between 0 to 1

In [ ]:
# plot both together to compare
fig, ax = plt.subplots(1, 2, figsize=(15, 3))
sns.histplot(original_data_ExistingUnits, ax=ax[0], kde=True, legend=False)
ax[0].set_title("Original Data")
sns.histplot(scaled_data_ExistingUnits, ax=ax[1], kde=True, legend=False)
ax[1].set_title("Scaled data")
plt.show()

## normalization

In [ ]:
postive_Estimated_Cost_index=cleanedPermit['Estimated Cost']>0
postiveCosts=cleanedPermit['Estimated Cost'].iloc[postive_Estimated_Cost_index]

In [ ]:
normalized_Estimated_Cost = pd.Series(stats.boxcox(postiveCosts)[0],  name='Estimated Cost Real', index=postiveCosts.index)

as seen in graph below data is now transformed or normalized . insted of concentraed distribution it is now normalized distribution

In [ ]:
# plot both together to compare
fig, ax=plt.subplots(1, 2, figsize=(15, 3))
sns.histplot(postiveCosts, ax=ax[0], kde=True, legend=False)
ax[0].set_title("Original Data")
sns.histplot(normalized_Estimated_Cost, ax=ax[1], kde=True, legend=False)
ax[1].set_title("Normalized data")
plt.show()

# Date Parsing

In [ ]:
cleanedPermit[['Current Status Date','Filed Date','Issued Date','Completed Date','First Construction Document Date']]

 after parsing dtype is changed to datetime so that we can use dt function directly on df

In [ ]:
cleanedPermit['Issued Date Parse']=pd.to_datetime(cleanedPermit['Issued Date'],format='%m/%d/%Y')

  clean data remove any integera or empty and replace with nan for making date fromat same

In [ ]:
cleanedPermit['Completed Date']=cleanedPermit['Completed Date'].replace(0,np.nan)

In [ ]:
cleanedPermit['Completed Date Parse']=pd.to_datetime(cleanedPermit['Completed Date'],format='%m/%d/%Y') 

 date plot

In [ ]:
day_of_month_issue = cleanedPermit['Issued Date Parse'].dt.day
day_of_month_complete = cleanedPermit['Completed Date Parse'].dt.day

# remove na's
day_of_month_issues = day_of_month_issue.dropna()
day_of_month_complete = day_of_month_complete.dropna()
 
# Combine data for plotting
plot_data = pd.DataFrame({
    'Day': pd.concat([day_of_month_issues, day_of_month_complete]),
    'Type': (['Issued'] * len(day_of_month_issues) + 
             ['Completed'] * len(day_of_month_complete))
})

# Overlay histogram (bars) and KDE (line)
sns.displot(data=plot_data, x='Day', hue='Type', 
            kind='hist', kde=True, 
            element='bars', alpha=0.4, 
            common_norm=False, bins=31)

plt.title('Distribution of Permit Issues vs Completions by Day of Month')
plt.show()

# Cleaning Data insconssitency

take neigborhoods as an example where names can be miss spelled

In [ ]:
permits['Neighborhoods - Analysis Boundaries'].unique().tolist()

normilze the data by 

1. convert to lower case
2. remove trailing white spaces

In [ ]:

permits['Neighborhoods - Analysis Boundaries'] = permits['Neighborhoods - Analysis Boundaries'].str.lower()

permits['Neighborhoods - Analysis Boundaries'] = permits['Neighborhoods - Analysis Boundaries'].str.strip()

In [ ]:
# we can check mathing based on fuzzy logic and based on mathcing perentage dicide to replace values
matches = fuzzywuzzy.process.extract("McLaren Park", permits['Neighborhoods - Analysis Boundaries'].tolist(), limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio)